# 6.5. 池化层
:label:`sec_pooling`

通常当我们处理图像时，我们希望逐渐降低隐藏表示的空间分辨率，聚集信息，这样的随着我们在神经网络中层叠的上升，每个神经元对其敏感的感受野（输入）就越大。

而我们的机器学习任务通常会跟全局图像的问题有关（例如，“图像是否包含一只猫呢？”）， 所以我们最后一层的神经元应该对整个输入的全局敏感。通过逐渐聚合信息，生成越来越粗糙的映射，最终实现学习全局表示的目标，同时将卷积图层的所有优势保留在中间层。

此外，当检测较底层的特征时（例如 :numref:`sec_conv_layer` 中所讨论的边缘），我们通常希望这些特征保持某种程度上的平移不变性。例如，如果我们拍摄黑白之间轮廓清晰的图像 `X`，并将整个图像向右移动一个像素，即 `Z[i, j] = X[i, j + 1]`，则新图像 `Z` 的输出可能大不相同。而在现实中，随着拍摄角度的移动，任何物体几乎不可能发生在同一像素上。即使用三脚架拍摄一个静止的物体，由于快门的移动而引起的相机振动，可能会使所有物体左右移动一个像素（除了高端相机配备了特殊功能来解决这个问题）。

本节将介绍 *池化*（pooling）层，它具有双重目的：降低卷积层对位置的敏感性，同时降低对空间降采样表示的敏感性。


## 6.5.1. 最大池化层和平均池化层

与卷积层类似，池化层运算符由一个固定形状的窗口组成，该窗口根据其步幅大小在输入的所有区域上滑动，为固定形状窗口（有时称为 *池化窗口*）遍历的每个位置计算一个输出。
然而，不同于卷积层中的输入与卷积核之间的互相关计算，池化层不包含参数。
相反，池运算符是确定性的，我们通常计算池化窗口中所有元素的最大值或平均值。这些操作分别称为 *最大池化层* （maximum pooling）和 *平均池化层* （average pooling）。

在这两种情况下，与互相关运算符一样，池化窗口从输入张量的左上角开始，从左到右、从上到下的在输入张量内滑动。在池化窗口到达的每个位置，它计算该窗口中输入子张量的最大值或平均值，具体取决于是使用了最大池化层还是平均池化层。
<center><img src='/Users/xutongtong/Desktop/pooling1.svg' ></center>


![池化窗口形状为 $2\times 2$ 的最大池化层。着色部分是第一个输出元素，以及用于计算这个输出的输入元素: $\max(0, 1, 3, 4)=4$.](../img/pooling.svg)
:label:`fig_pooling`

:numref:`fig_pooling` 中的输出张量的高度为 $2$，宽度为 $2$。这四个元素为每个池化窗口中的最大值：

$$
\max(0, 1, 3, 4)=4,\\
\max(1, 2, 4, 5)=5,\\
\max(3, 4, 6, 7)=7,\\
\max(4, 5, 7, 8)=8.\\
$$

池化窗口形状为 $p \times q$ 的池化层称为 $p \times q$ 池化层，池化操作称为 $p \times q$ 池化。

回到本节开头提到的对象边缘检测示例，现在我们将使用卷积层的输出作为 $2\times 2$ 最大池化的输入。
设置卷积层输入为 `X`，池化层输出为 `Y`。
无论 `X[i, j]` 和 `X[i, j + 1]` 的值是否不同，或 `X[i, j + 1]` 和 `X[i, j + 2]` 的值是否不同，池化层始终输出 `Y[i, j] = 1`。
也就是说，使用 $2\times 2$ 最大池化层，即使在高度或宽度上移动一个元素，卷积层仍然可以识别到模式。

在下面的代码中的 `pool2d` 函数，我们(**实现池化层的正向传播**)。
此功能类似于 :numref:`sec_conv_layer` 中的 `corr2d` 函数。
然而，这里我们没有卷积核，输出为输入中每个区域的最大值或平均值。


In [2]:
import paddle

ModuleNotFoundError: No module named 'paddle'

In [9]:
def pool2d(X, pool_size, mode='max'):
    p_h, p_w = pool_size
    Y = paddle.zeros([X.shape[0] - p_h + 1, X.shape[1] - p_w + 1])
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i, j] = X[i: i + p_h, j: j + p_w].max()
            elif mode == 'avg':
                Y[i, j] = X[i: i + p_h, j: j + p_w].mean()       
    return Y

我们可以构建 :numref:`fig_pooling` 中的输入张量 `X`，[**验证二维最大池化层的输出**]。


In [10]:
X = paddle.to_tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]], dtype="float32")
pool2d(X, (2, 2))

Tensor(shape=[2, 2], dtype=float32, place=CUDAPlace(0), stop_gradient=True,
       [[4., 5.],
        [7., 8.]])

此外，我们还可以(**验证平均池化层**)。

In [11]:
pool2d(X, (2, 2), 'avg')

Tensor(shape=[2, 2], dtype=float32, place=CUDAPlace(0), stop_gradient=True,
       [[2., 3.],
        [5., 6.]])

## **6.5.2. 填充和步幅**

与卷积层一样，池化层也可以改变输出形状。和以前一样，我们可以通过填充和步幅以获得所需的输出形状。
下面，我们用深度学习框架中内置的二维最大池化层，来演示池化层中填充和步幅的使用。
我们首先构造了一个输入张量 `X`，它有四个维度，其中样本数和通道数都是 1。


In [14]:
X = paddle.arange(16, dtype="float32").reshape((1, 1, 4, 4))
X

Tensor(shape=[1, 1, 4, 4], dtype=float32, place=CUDAPlace(0), stop_gradient=True,
       [[[[0. , 1. , 2. , 3. ],
          [4. , 5. , 6. , 7. ],
          [8. , 9. , 10., 11.],
          [12., 13., 14., 15.]]]])

默认情况下，(**深度学习框架中的步幅与池化窗口的大小相同**)。
因此，如果我们使用形状为 `(3, 3)` 的池化窗口，那么默认情况下，我们得到的步幅形状为 `(3, 3)`。


In [16]:
pool2d = paddle.nn.MaxPool2D(3, stride=3)
pool2d(X)

Tensor(shape=[1, 1, 1, 1], dtype=float32, place=CUDAPlace(0), stop_gradient=True,
       [[[[10.]]]])

**填充和步幅可以手动设定**。

In [17]:
pool2d = paddle.nn.MaxPool2D(3, padding=1, stride=2)
pool2d(X)

Tensor(shape=[1, 1, 2, 2], dtype=float32, place=CUDAPlace(0), stop_gradient=True,
       [[[[5. , 7. ],
          [13., 15.]]]])

当然，我们可以(**设定一个任意大小的矩形池化窗口，并分别设定填充和步幅的高度和宽度**)。


In [18]:
pool2d = paddle.nn.MaxPool2D((2, 4), padding=(1, 2), stride=(2, 3))
pool2d(X)

Tensor(shape=[1, 1, 3, 2], dtype=float32, place=CUDAPlace(0), stop_gradient=True,
       [[[[1. , 3. ],
          [9. , 11.],
          [13., 15.]]]])

## 6.5.3. 多个通道

在处理多通道输入数据时，[**池化层在每个输入通道上单独运算**]，而不是像卷积层一样在通道上对输入进行汇总。
这意味着池化层的输出通道数与输入通道数相同。
下面，我们将在通道维度上连结张量 `X` 和 `X + 1`，以构建具有 2 个通道的输入。

In [19]:
X = paddle.concat((X, X + 1), axis=1)
X

Tensor(shape=[1, 2, 4, 4], dtype=float32, place=CUDAPlace(0), stop_gradient=True,
       [[[[0. , 1. , 2. , 3. ],
          [4. , 5. , 6. , 7. ],
          [8. , 9. , 10., 11.],
          [12., 13., 14., 15.]],

         [[1. , 2. , 3. , 4. ],
          [5. , 6. , 7. , 8. ],
          [9. , 10., 11., 12.],
          [13., 14., 15., 16.]]]])

如下所示，池化后输出通道的数量仍然是 2。


In [21]:
pool2d = paddle.nn.MaxPool2D(3, padding=1, stride=2)
pool2d(X)

Tensor(shape=[1, 2, 2, 2], dtype=float32, place=CUDAPlace(0), stop_gradient=True,
       [[[[5. , 7. ],
          [13., 15.]],

         [[6. , 8. ],
          [14., 16.]]]])

## 6.5.4. 小结

* 对于给定输入元素，最大池化层会输出该窗口内的最大值，平均池化层会输出该窗口内的平均值。
* 池化层的主要优点之一是减轻卷积层对位置的过度敏感。
* 我们可以指定池化层的填充和步幅。
* 使用最大池化层以及大于 1 的步幅，可减少空间维度（如高度和宽度）。
* 池化层的输出通道数与输入通道数相同。


## 6.5.3. 练习

1. 你能将平均池化层作为卷积层的特殊情况实现吗？
1. 你能将最大池化层作为卷积层的特殊情况实现吗？
1. 假设池化层的输入大小为 $c\times h\times w$，则池化窗口的形状为 $p_h\times p_w$，填充为 $(p_h, p_w)$，步幅为 $(s_h, s_w)$。这个池化层的计算成本是多少？
1. 为什么最大池化层和平均池化层的工作方式不同？
1. 我们是否需要最小池化层？可以用已知函数替换它吗？
1. 除了平均池化层和最大池化层，是否有其它函数可以考虑（提示：回忆 `softmax` ）？为什么它可能不受欢迎？
